In [192]:
%tensorflow_version  1.x
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

## 데이터 로드(MNIST)

In [193]:
import random
from tensorflow.examples.tutorials.mnist import input_data

## 사용할 GPU설정, 프로그램이 첫 번째 GPU를 사용하도록 제한

In [194]:
tf.debugging.set_log_device_placement(True) 

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    print(e)

### 학습 자료에선 tf.set_random_seed() 라고 써있음 
버전이 바뀌면서 사용방법이 달라짐

In [195]:
tf.set_random_seed(202131036)

## 데이터 로드

In [196]:
learning_rate = 0.001
training_epoch = 15
batch_size = 100

In [197]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Placeholder : 입력 받는 값을 실수형태로 입력받겠다. 

In [198]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1,28,28,1])
y = tf.placeholder(tf.float32, [None, 10])

## 모델
#### conv layer1 

In [199]:
W1 = tf.Variable(tf.random_normal([3,3,1, 32], stddev=0.01)) 
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = "SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME')

tf.random_normal(shape, stddev): shape의 모양으로 정규분포로부터 랜덤한 값을 출력함

stddev : 	A Tensor or Python value of type dtype, broadcastable with mean. The standard deviation of the normal distribution.



w1 : filters 

filters , kernel shape = [fileter_height, filter_width, in_channels, out_channels]

**L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = "SAME")**
+ batch_size = 100
+ x_img = [28,28,1]
+ W1 shape = [3,3,1,32]
+ Flatten shape = [3*3*1, 32] = [9,32]
+ Extract shape patches = [100, 28,28, 9]
+ 각 patch당 output = [28, 28, 32]

**L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME')**
+ L1 = [28,28,32]
+ output = [14,14,32]




---



In [200]:
W2 = tf.Variable(tf.random_normal([3,3,1,64], stddev = 0.01))
L2 = tf.nn.conv2d(X_img, W2, strides = [1,1,1,1], padding = "SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")
L2_flat = tf.reshape(L2, [-1, 7*7*64]) 

**L2 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = "SAME")**
+ X_img = [14,14,1] 
+ W2 = [3,3,1,64]
+ L2 output = [14,14,64] 

**L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")**
+ L2 output = [14,14, 64]

**L2_flat = tf.reshape(L2, [-1, 7x7x64])**
+ L2_flat = [1, 7x7x64]




In [201]:
W3 = tf.get_variable("W3", shape=[7*7*64, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

tf.contrib.layers.xavier_initailize()

성능을 조금더 높이는 방법은 랜덤으로 초기값을 잡아왔던 W(weight)값을 잘 건드려 주기 위해 xavier사용 

In [202]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

batpropagation will happen into both logits and labels.

+ logits : 각 label[i]에 대한 activation 

In [203]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



tf.global_variables_initializer() : 
Returns an Op that initializes global variables.

In [204]:
for epoch in range(training_epoch):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples/ batch_size)

  for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      feed_dict = {X: batch_xs, y:batch_ys}
      c,_ = sess.run([cost, optimizer], feed_dict=feed_dict)
      avg_cost += c/total_batch #평균 cost

  print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))
print('learning finished')

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X:mnist.test.images, y:mnist.test.labels}))

In [ ]:
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
print('Prediction: ', sess.run(tf.argmax(logits, 1), feed_dict={X:mnist.test.images[r:r+1]}))

In [ ]:
sizetest = tf.constant([[1, 1, 1, 1], [2, 2, 2, 2,]])
size = tf.size(sizetest)
shape = tf.shape(sizetest)
with tf.Session() as sess:
    s_shape = sess.run(shape)
    print(s_shape)
    print(s_shape[0])
    print(s_shape[1])
    